In [ ]:
%%sh

# The name of our algorithm
algorithm_name=lstm_gender_classifier

cd container

chmod +x decision_trees/train
chmod +x decision_trees/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.

aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

In [1]:
# S3 prefix
prefix = 'lstm-gender-classifier'

# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

## Create the session

The session remembers our connection parameters to SageMaker. We'll use it to perform all of our SageMaker operations.

In [2]:
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

## Upload the data for training

In [3]:
WORK_DIRECTORY = 'data'

data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)

INFO:sagemaker:Created S3 bucket: sagemaker-us-east-1-741855114961


## Create an estimator and fit the model

In [4]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/lstm-gender-classifier:latest'.format(account, region)

tree = sage.estimator.Estimator(image,
                       role, 1, 'ml.c4.8xlarge',
                       output_path="s3://{}/output".format(sess.default_bucket()),
                       sagemaker_session=sess)

tree.fit(data_location)

INFO:sagemaker:Creating training-job with name: lstm-gender-classifier-2018-03-12-04-16-47-095


.............................................................
/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
+-ml/
  |
  +-input/
  | |
  | +-config/
  | | |
  | | +-metric-definition-regex.json
  | | |
  | | +-hyperparameters.json
  | | |
  | | +-resourceconfig.json
  | | |
  | | +-init-config.json
  | | |
  | | +-inputdataconfig.json
  | |
  | +-data/
  |   |
  |   +-training-manifest
  |   |
  |   +-training-manifest-1
  |   |
  |   +-training/
  |     |
  |     +-allnames.txt
  |     |
  |     +-names.txt
  |
  +-output/
  | |
  | +-data/
  |
  +-model/
Starting the training.
Heperparameters file : {}
Hyperparameters initialized
Loading data from : /opt/ml/input/data/training/allnames.txt
Training data l

85356/85356 [==============================] - 69s 808us/step - loss: 0.6336 - acc: 0.6800 - val_loss: 0.5365 - val_acc: 0.7285
Epoch 2/5
84000/85356 [============================>.] - ETA: 1s - loss: 0.5461 - acc: 0.7254


85356/85356 [==============================] - 68s 797us/step - loss: 0.5455 - acc: 0.7258 - val_loss: 0.5168 - val_acc: 0.7437
Epoch 3/5
85356/85356 [==============================] - 68s 796us/step - loss: 0.5172 - acc: 0.7442 - val_loss: 0.4895 - val_acc: 0.7595
Epoch 4/5
 4000/85356 [>.............................] - ETA: 1:00 - loss: 0.4939 - acc: 0.7595


85356/85356 [==============================] - 68s 796us/step - loss: 0.4888 - acc: 0.7633 - val_loss: 0.4717 - val_acc: 0.7750
Epoch 5/5
 5000/85356 [>.............................] - ETA: 59s - loss: 0.4912 - acc: 0.7618 


85356/85356 [==============================] - 68s 796us/step - loss: 0.4716 - acc: 0.7747 - val_loss: 0.4749 - val_acc: 0.7620
Model trained
Training complete.
===== Job Complete =====


## Deploy the model

Deploying the model to SageMaker hosting just requires a `deploy` call on the fitted model. This call takes an instance count, instance type, and optionally serializer and deserializer functions. These are used when the resulting predictor is created on the endpoint.

In [5]:
from sagemaker.predictor import csv_serializer
predictor = tree.deploy(1, 'ml.m4.xlarge', serializer=csv_serializer)

INFO:sagemaker:Creating model with name: lstm-gender-classifier-2018-03-12-04-29-58-528
INFO:sagemaker:Creating endpoint with name lstm-gender-classifier-2018-03-12-04-16-47-095


----------------------------------------------------------------------------------------!

## Choose some data and use it for a prediction

Prediction is as easy as calling predict with the predictor we got back from deploy and the data we want to do predictions with. The serializers take care of doing the data conversions for us.

In [7]:
lines = [line.rstrip('\n') for line in open('container/local_test/names.txt', 'r')]
for line in lines:
    print(predictor.predict(line))

b'M,M,F,F,'


## Optional cleanup

When you're done with the endpoint, you'll want to clean it up.

In [8]:
sess.delete_endpoint(predictor.endpoint)

INFO:sagemaker:Deleting endpoint with name: lstm-gender-classifier-2018-03-12-04-16-47-095
